# Setup

## Load libraries

In [ ]:
library(tidyverse)
library(lubridate)

## Load datasets

In [ ]:
# Load data
if (!exists("loans")) loans <- readRDS("datasets/lending_club_reformatted_paid.rds")
if (!exists("loansIRR"))
  loansIRR <- read.csv("datasets/LoanIRR.csv") %>% as_tibble()

if (!exists("loansCreditMargin"))
  loansCreditMargin <- read.csv("datasets/CreditMargins.csv") %>% as_tibble() %>%
    # Remove monthDefault since already in IRR dataframe
    select(-monthDefault)

nSamples <- nrow(loans)


loansWorkingSet <- loans %>%
  # add IRR and credit margins calculations
  left_join(loansIRR, by = "loanID") %>%
  left_join(loansCreditMargin, by = "loanID") %>%

  # add a % principal loss variable
  mutate(principal_loss_pct = (funded_amnt - total_rec_prncp) / funded_amnt)

## Import the descriptions of all the variables

This description was generated from the spreadsheet included in the Kaggle dataset, with further addition and generated as R code.

This is a copy from 01-startup/Rmd

In [ ]:
LC_variable <-
  tibble(variable_name = 'loanID',
         description = 'NOTE THIS IS NOT AN ORIGINAL VARIABLE. IT WAS ADDED FOR THE PURPOSE OF TRACKING LOANS INDIVIDUALLY AS AND WHEN NEEDED.',
         inModel = TRUE, inPrediction = FALSE, inPricing = FALSE) %>%
  add_row(variable_name = 'loan_amnt' ,
          description = 'The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.',
          inModel = TRUE, inPrediction = FALSE, inPricing = FALSE) %>%
  add_row(
    variable_name = 'funded_amnt' ,
    description = 'The total amount committed to that loan at that point in time.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'funded_amnt_inv' ,
    description = 'The total amount committed by investors for that loan at that point in time.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'term' ,
    description = 'The number of payments on the loan. Values are in months and can be either 36 or 60.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'int_rate' ,
    description = 'Interest Rate on the loan',
    inModel = TRUE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'installment' ,
    description = 'The monthly payment owed by the borrower if the loan originates.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'grade' ,
    description = 'LC assigned loan grade',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sub_grade' ,
    description = 'LC assigned loan subgrade',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'emp_title' ,
    description = 'The job title supplied by the Borrower when applying for the loan.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'emp_length' ,
    description = 'Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. ',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'home_ownership' ,
    description = 'The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER, NONE',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'annual_inc' ,
    description = 'The self-reported annual income provided by the borrower during registration. NOT USED AS A VARIABLE SINCE JOINT INCOME ALREADY INCLUDES IT.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'verification_status' ,
    description = 'Indicates if income was verified by LC, not verified, or if the income source was verified',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'issue_d' ,
    description = 'The month which the loan was funded',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'loan_status' ,
    description = 'Current status of the loan',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'pymnt_plan' ,
    description = 'Indicates if a payment plan has been put in place for the loan',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'url' ,
    description = 'URL for the LC page with listing data.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'desc' ,
    description = 'Loan description provided by the borrower',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'purpose' ,
    description = 'A category provided by the borrower for the loan request. ',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'title' ,
    description = 'The loan title provided by the borrower',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'zip_code' ,
    description = 'The first 3 numbers of the zip code provided by the borrower in the loan application.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'addr_state' ,
    description = 'The state provided by the borrower in the loan application',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'dti' ,
    description = 'A ratio calculated using the borrower s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower s self-reported monthly income. NOT USED AS A VARIABLE. ONLY USE JOINT DTI.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'delinq_2yrs' ,
    description = 'The number of 30+ days past-due incidences of delinquency in the borrower s credit file for the past 2 years',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'earliest_cr_line' ,
    description = 'The month the borrower s earliest reported credit line was opened',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'inq_last_6mths' ,
    description = 'The number of inquiries in past 6 months (excluding auto and mortgage inquiries)',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mths_since_last_delinq' ,
    description = 'The number of months since the borrower s last delinquency.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mths_since_last_record' ,
    description = 'The number of months since the last public record.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'open_acc' ,
    description = 'The number of open credit lines in the borrower s credit file.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'pub_rec' ,
    description = 'Number of derogatory public records',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'revol_bal' ,
    description = 'Total credit revolving balance',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'revol_util' ,
    description = 'Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'total_acc' ,
    description = 'The total number of credit lines currently in the borrower s credit file',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'initial_list_status' ,
    description = 'The initial listing status of the loan. Possible values are – W, F',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'out_prncp' ,
    description = 'Remaining outstanding principal for total amount funded. NOTE ONCE A LOAN IS REPAID OR CHARGED OFF, THIS AMOUNT BECOMES 0. ',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'out_prncp_inv' ,
    description = 'Remaining outstanding principal for portion of total amount funded by investors. NOTE ONCE A LOAN IS REPAID OR CHARGED OFF, THIS AMOUNT BECOMES 0. ',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'total_pymnt' ,
    description = 'Payments received to date for total amount funded',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'total_pymnt_inv' ,
    description = 'Payments received to date for portion of total amount funded by investors',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'total_rec_prncp' ,
    description = 'Principal received to date. NOTE THIS AMOUNT WILL SHOW WHETHER A BORROWER DID NOT REPAY IN FULL',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'total_rec_int' ,
    description = 'Interest received to date',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'total_rec_late_fee' ,
    description = 'Late fees received to date',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'recoveries' ,
    description = 'Post charge off gross recovery. NOTE IF A LOAN IS REPAID, THIS AMOUNT IS 0. ',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'collection_recovery_fee' ,
    description = 'Post charge off collection fee',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'last_pymnt_d' ,
    description = 'Last month payment was received',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'last_pymnt_amnt' ,
    description = 'Last total payment amount received',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'next_pymnt_d' ,
    description = 'Next scheduled payment date',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'last_credit_pull_d' ,
    description = 'The most recent month LC pulled credit for this loan',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'collections_12_mths_ex_med' ,
    description = 'Number of collections in 12 months excluding medical collections',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mths_since_last_major_derog' ,
    description = 'Months since most recent 90-day or worse rating',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'policy_code' ,
    description = 'Publicly available policy_code=1 / New products not publicly available policy_code=2',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'application_type' ,
    description = 'Indicates whether the loan is an individual application or a joint application with two coborrowers',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'annual_inc_joint' ,
    description = 'The combined self-reported annual income provided by the coborrowers during registration',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'dti_joint' ,
    description = 'A ratio calculated using the coborrowers total monthly payments on the total debt obligations, excluding mortgages and the requested LC loan, divided by the coborrowers combined self-reported monthly income',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'verification_status_joint' ,
    description = 'Indicates if income was verified by LC, not verified, or if the income source was verified',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'acc_now_delinq' ,
    description = 'The number of accounts on which the borrower is now delinquent.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'tot_coll_amt' ,
    description = 'Total collection amounts ever owed',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'tot_cur_bal' ,
    description = 'Total current balance of all accounts',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'open_acc_6m' ,
    description = 'Number of open trades in last 6 months',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'open_act_il' ,
    description = 'Number of currently active installment trades',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'open_il_12m' ,
    description = 'Number of installment accounts opened in past 12 months',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'open_il_24m' ,
    description = 'Number of installment accounts opened in past 24 months',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'mths_since_rcnt_il' ,
    description = 'Months since most recent instalment accounts opened',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'total_bal_il' ,
    description = 'Total current balance of all installment accounts',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'il_util' ,
    description = 'Ratio of total current balance to high credit/credit limit on all install acct',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'open_rv_12m' ,
    description = 'Number of revolving trades opened in past 12 months',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'open_rv_24m' ,
    description = 'Number of revolving trades opened in past 24 months',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'max_bal_bc' ,
    description = 'Maximum current balance owed on all revolving accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'all_util' ,
    description = 'Balance to credit limit on all trades',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'total_rev_hi_lim' ,
    description = 'Total revolving high credit/credit limit',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'inq_fi' ,
    description = 'Number of personal finance inquiries',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'total_cu_tl' ,
    description = 'Number of finance trades',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'inq_last_12m' ,
    description = 'Number of credit inquiries in past 12 months',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'acc_open_past_24mths' ,
    description = 'Number of trades opened in past 24 months.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'avg_cur_bal' ,
    description = 'Average current balance of all accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'bc_open_to_buy' ,
    description = 'Total open to buy on revolving bankcards.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'bc_util' ,
    description = 'Ratio of total current balance to high credit/credit limit for all bankcard accounts.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'chargeoff_within_12_mths' ,
    description = 'Number of charge-offs within 12 months',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'delinq_amnt' ,
    description = 'The past-due amount owed for the accounts on which the borrower is now delinquent.',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'mo_sin_old_il_acct' ,
    description = 'Months since oldest bank instalment account opened',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mo_sin_old_rev_tl_op' ,
    description = 'Months since oldest revolving account opened',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mo_sin_rcnt_rev_tl_op' ,
    description = 'Months since most recent revolving account opened',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mo_sin_rcnt_tl' ,
    description = 'Months since most recent account opened',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mort_acc' ,
    description = 'Number of mortgage accounts.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mths_since_recent_bc' ,
    description = 'Months since most recent bankcard account opened.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mths_since_recent_bc_dlq' ,
    description = 'Months since most recent bankcard delinquency',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mths_since_recent_inq' ,
    description = 'Months since most recent inquiry.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'mths_since_recent_revol_delinq' ,
    description = 'Months since most recent revolving delinquency.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_accts_ever_120_pd' ,
    description = 'Number of accounts ever 120 or more days past due',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_actv_bc_tl' ,
    description = 'Number of currently active bankcard accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_actv_rev_tl' ,
    description = 'Number of currently active revolving trades',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_bc_sats' ,
    description = 'Number of satisfactory bankcard accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_bc_tl' ,
    description = 'Number of bankcard accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_il_tl' ,
    description = 'Number of installment accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_op_rev_tl' ,
    description = 'Number of open revolving accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_rev_accts' ,
    description = 'Number of revolving accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_rev_tl_bal_gt_0' ,
    description = 'Number of revolving trades with balance >0',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_sats' ,
    description = 'Number of satisfactory accounts',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_tl_120dpd_2m' ,
    description = 'Number of accounts currently 120 days past due (updated in past 2 months)',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_tl_30dpd' ,
    description = 'Number of accounts currently 30 days past due (updated in past 2 months)',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_tl_90g_dpd_24m' ,
    description = 'Number of accounts 90 or more days past due in last 24 months',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'num_tl_op_past_12m' ,
    description = 'Number of accounts opened in past 12 months',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'pct_tl_nvr_dlq' ,
    description = 'Percent of trades never delinquent',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'percent_bc_gt_75' ,
    description = 'Percentage of all bankcard accounts > 75% of limit.',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'pub_rec_bankruptcies' ,
    description = 'Number of public record bankruptcies',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'tax_liens' ,
    description = 'Number of tax liens',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'tot_hi_cred_lim' ,
    description = 'Total high credit/credit limit',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'total_bal_ex_mort' ,
    description = 'Total credit balance excluding mortgage',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'total_bc_limit' ,
    description = 'Total bankcard high credit/credit limit',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'total_il_high_credit_limit' ,
    description = 'Total installment high credit/credit limit',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'revol_bal_joint' ,
    description = 'Total credit revolving balance',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_earliest_cr_line' ,
    description = 'Earliest credit line at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_inq_last_6mths' ,
    description = 'Credit inquiries in the last 6 months at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_mort_acc' ,
    description = 'Number of mortgage accounts at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_open_acc' ,
    description = 'Number of open trades at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_revol_util' ,
    description = 'Ratio of total current balance to high credit/credit limit for all revolving accounts. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_open_act_il' ,
    description = 'Number of currently active installment trades at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_num_rev_accts' ,
    description = 'Number of revolving accounts at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_chargeoff_within_12_mths' ,
    description = 'Number of charge-offs within last 12 months at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_collections_12_mths_ex_med' ,
    description = 'Number of collections within last 12 months excluding medical collections at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'sec_app_mths_since_last_major_derog' ,
    description = 'Months since most recent 90-day or worse rating at time of application for the secondary applicant. VARIABLE NOT USED. WE RELY ON THE MAIN BORROWER IN THE FIRST INSTANCE.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'hardship_flag' ,
    description = 'Flags whether or not the borrower is on a hardship plan',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_type' ,
    description = 'Describes the hardship plan offering',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'hardship_reason' ,
    description = 'Describes the reason the hardship plan was offered',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'hardship_status' ,
    description = 'Describes if the hardship plan is active, pending, cancelled, completed, or broken',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'deferral_term' ,
    description = 'Amount of months that the borrower is expected to pay less than the contractual monthly payment amount due to a hardship plan',
    inModel = FALSE,
    inPrediction = TRUE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_amount' ,
    description = 'The interest payment that the borrower has committed to make each month while they are on a hardship plan',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_start_date' ,
    description = 'The start date of the hardship plan period',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_end_date' ,
    description = 'The end date of the hardship plan period',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'payment_plan_start_date' ,
    description = 'The day the first hardship plan payment is due. For example, if a borrower has a hardship plan period of 3 months, the start date is the start of the three-month period in which the borrower is allowed to make interest-only payments.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_length' ,
    description = 'The number of months the borrower will make smaller payments than normally obligated due to a hardship plan',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_dpd' ,
    description = 'Account days past due as of the hardship plan start date',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_loan_status' ,
    description = 'Loan Status as of the hardship plan start date',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'orig_projected_additional_accrued_interest' ,
    description = 'The original projected additional interest amount that will accrue for the given hardship payment plan as of the Hardship Start Date. This field will be null if the borrower has broken their hardship payment plan.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_payoff_balance_amount' ,
    description = 'The payoff balance amount as of the hardship plan start date',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'hardship_last_payment_amount' ,
    description = 'The last payment amount as of the hardship plan start date',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = TRUE
  ) %>%
  add_row(
    variable_name = 'disbursement_method' ,
    description = 'The method by which the borrower receives their loan. Possible values are: CASH, DIRECT_PAY',
    inModel = TRUE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'debt_settlement_flag' ,
    description = 'Flags whether or not the borrower, who has charged-off, is working with a debt-settlement company.',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'debt_settlement_flag_date' ,
    description = 'The most recent date that the Debt_Settlement_Flag has been set  ',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'settlement_status' ,
    description = 'The status of the borrower’s settlement plan. Possible values are: COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAFT',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'settlement_date' ,
    description = 'The date that the borrower agrees to the settlement plan',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'settlement_amount' ,
    description = 'The loan amount that the borrower has agreed to settle for',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(
    variable_name = 'settlement_percentage' ,
    description = 'The settlement amount as a percentage of the payoff balance amount on the loan',
    inModel = FALSE,
    inPrediction = FALSE,
    inPricing = FALSE
  ) %>%
  add_row(variable_name = 'settlement_term' ,
          description = 'The number of months that the borrower will be on the settlement plan',
          inModel = FALSE,
          inPrediction = FALSE,
          inPricing = FALSE)


In [ ]:
# Select the variables checked in 01-startup.Rmd
varList <- c(LC_variable[LC_variable$inModel == TRUE, "variable_name"])$variable_name
varList <- c(varList, "grade_num", "sub_grade_num", "principal_loss_pct", "creditMargin", "monthDefault")

# Make sure that some variables are NOT in included in the training set
varRemove <- c("")

In [ ]:
#################################################################################################
##
## Prepare a dataset with ONLY the predictors
##
loansPredictors <-
  loansWorkingSet %>%

  # Keep the chosen predictors
  # Use one_of() to avoid errors if column does not exist
  select(one_of(varList)) %>%
  select(-one_of(varRemove)) %>%

  # [TODO] FOR THE MOMENT UNTIL MACRODATA IS FIXED
  select(-one_of("addr_state")) %>%

  ##
  ## Dates to numeric, in 'decimal' years since 2000
  ##
  mutate_at(c("issue_d", "earliest_cr_line"), function(d) {
    return(year(d) - 2000 + (month(d) - 1) / 12)
  }) %>%

  ## Add polynomials of the dates to model the time-trend shape
  mutate(issue_d2 = issue_d ^ 2,
         issue_d3 = issue_d ^ 3,
         earliest_cr_line2 = earliest_cr_line ^ 2,
         earliest_cr_line3 = earliest_cr_line ^ 3) %>%

  # Fill missing values
  mutate_all(~replace(., is.na(.), 0))

In [ ]:
#################################################################################################
##
## One-hot encoding of factors (in case model do not mix continuous and cat. variables)
##
modelX <-
  loansPredictors %>%

  # Binary categories into 0/1
  mutate(disbursement_method = if_else(disbursement_method == "Cash", 1, 0)) %>%
  rename(cashDisbursement = disbursement_method) %>%

  mutate(application_type = if_else(application_type == "Individual", 0, 1)) %>%
  rename(applicationJoint = application_type) %>%

  ## Widen the categorical datas
  # Verification
  mutate(verifiedSource      = if_else(verification_status       == "Source Verified", 1, 0),
         verified            = if_else(verification_status       == "Not Verified", 0, 1),
         verifiedJointSource = if_else(verification_status_joint == "Source Verified", 1, 0),
         verifiedJoint       = if_else(verification_status_joint %in% c("", "Not Verified"), 0, 1)) %>%
  select(-verification_status, -verification_status_joint) %>%

  # Home ownership
  mutate(homeANY = if_else(home_ownership == "ANY", 1, 0),
         homeMORTGAGE = if_else(home_ownership == "MORTGAGE", 1, 0),

         # Delete beacuse useless
         homeNONE = if_else(home_ownership == "NONE", 1, 0),
         homeOTHER = if_else(home_ownership == "OTHER", 1, 0),
         homeOWN = if_else(home_ownership == "OWN", 1, 0),
         homeRENT = if_else(home_ownership == "RENT", 1, 0)) %>%
  select(-home_ownership) %>%

  # Purpose
  mutate(purpCAR = if_else(purpose == "car", 1, 0),
         purpCREDITCARD = if_else(purpose == "credit_card", 1, 0),
         purpCDEBTCONSO = if_else(purpose == "debt_consolidation", 1, 0),
         purpEDUCATION = if_else(purpose == "educational", 1, 0),
         purpHOMEIMPROV = if_else(purpose == "home_improvement", 1, 0),
         purpHOUSE = if_else(purpose == "house", 1, 0),
         purpPURCHASE = if_else(purpose == "major_purchase", 1, 0),
         purpMEDICAL = if_else(purpose == "medical", 1, 0),
         purpMOVING = if_else(purpose == "moving", 1, 0),
         purpOTHER = if_else(purpose == "other", 1, 0),
         purpRENEWABLE = if_else(purpose == "renewable_energy", 1, 0),
         purpSMALLBUS = if_else(purpose == "small_business", 1, 0),
         purpVACATION = if_else(purpose == "vacation", 1, 0),
         purpWEDDING = if_else(purpose == "wedding", 1, 0)) %>%
  select(-purpose)

Create training / test sets 80%/20%

In [ ]:
proportionTraining <- 0.8
set.seed(42)
sampleTraining  <- sample(1:nSamples, floor(nSamples * proportionTraining), replace = FALSE)

loansTraining <- loansPredictors %>% slice( sampleTraining)
loansTest <-     loansPredictors %>% slice(-sampleTraining)

modelTraining <- modelX %>% slice( sampleTraining)
modelTest <-     modelX %>% slice(-sampleTraining)

nSamplesTraining <- nrow(loansTraining)

Subsets of the training set containing 0.1%, 0.5% and 10% of the original set.


In [ ]:
set.seed(42 + 1001)
sample001 <- sample(1:nSamplesTraining,  floor(nSamplesTraining * 0.001), replace = FALSE)
loans001  <- loansTraining %>% slice(sample001)
model001  <- modelX %>% slice(sampleTraining) %>% slice(sample001)
G001      <- modelX %>% select(sub_grade_num) %>% slice(sample001)
loss001   <- loansTraining %>% select(principal_loss_pct) %>% slice(sample001)
CM001     <- loansTraining %>% select(creditMargin) %>% slice(sample001)

set.seed(42 + 1005)
sample005 <- sample(1:nSamplesTraining,  floor(nSamplesTraining * 0.005), replace = FALSE)
loans005  <- loansTraining %>% slice(sample005)
model005 <- modelX %>% slice(sampleTraining) %>% slice(sample005)
G005     <- modelX %>% select(sub_grade_num) %>% slice(sample005)
loss005  <- loansTraining %>% select(principal_loss_pct) %>% slice(sample005)
CM005    <- loansTraining %>% select(creditMargin) %>% slice(sample005)

set.seed(42 + 1100)
sample100 <- sample(1:nSamplesTraining,  floor(nSamplesTraining * 0.100), replace = FALSE)
loans100  <- loansTraining %>% slice(sample005)
model100 <- modelX %>% slice(sampleTraining) %>% slice(sample100)
G100     <- modelX %>% select(sub_grade_num) %>% slice(sample100)
loss100  <- loansTraining %>% select(principal_loss_pct) %>% slice(sample100)
CM100    <- loansTraining %>% select(creditMargin) %>% slice(sample100)

In [ ]:
nrow(CM001)
nrow(CM005)
nrow(CM100)

In [ ]:
names(loansWorkingSet)

In [ ]:
loansWorkingSet %>%
  sample_n(50000) %>% 
  filter(!is.na(creditMargin)) %>% 
  ggplot(aes(sub_grade, creditMargin)) +
  geom_boxplot() 

What are the values around 600%?

In [ ]:
tmp <- 
  loansWorkingSet %>%
  sample_n(50000) %>% 
  filter(!is.na(creditMargin)) %>% 
  filter(between(creditMargin, 6 * (1-0.10), 6 * (1 + 0.10))) 

In [ ]:
print(tmp)

In [ ]:
#library(cluster)
#library(factoextra)
#library(caret)
#library(xgboost)
#library(doParallel)

# Attach again, otherwise slice is masked
#library(dplyr)

# Extreme Gradient Boosting

This is carried out on a smaller sample (1 thousandth). 

In [ ]:
loansTmp <- model005 %>% select(-creditMargin, -principal_loss_pct, -monthDefault)
names(loansTmp)


In [ ]:
##############################################################
## 
## EXTREME GRADIENT BOOSTING
## 

{
    require(doParallel)
    require(caret)
    require(xgboost)
    
    tictoc::tic()
    cl <- makePSOCKcluster(4)
    registerDoParallel(cl)

    trainXGB <- train(x = loansTmp, y = CM005$creditMargin,
                      method="xgbTree",
                      trControl = trainControl(method = "cv"))

    stopCluster(cl)
    tictoc::toc()

    print(trainXGB)
    varImp(trainXGB)
    
    saveRDS(trainXGB, "datasets/XB_005_CreditMargin.rds")
}

In [ ]:
testTmp <- modelTest %>% sample_n(size = 0.005 * nrow(modelTest))

predictXGB <- predict(trainXGB, testTmp)
summary(predictXGB)

cat("RMSE = ", sqrt(sum((predictXGB - testTmp$creditMargin)^2) / nrow(testTmp)))

# K-Means

In [ ]:
loansTmp <- model001 %>% select(-creditMargin, -principal_loss_pct, -monthDefault)
names(loansTmp)

In [ ]:
##############################################################
## 
## K-MEANS
## 

{
    require(doParallel)
    require(caret)
    require(cluster)
    
    tictoc::tic()
    cl <- makePSOCKcluster(4)
    registerDoParallel(cl)

    trainKMEANS <- train(x = loansTmp, y = CM001$creditMargin,
                      method="knn",
                      tuneGrid = data.frame(k = seq(30, 50, 2)),
                      trControl = trainControl(method = "cv"))

    stopCluster(cl)
    tictoc::toc()

    #confusionMatrix(predict(trainXGB, val_test_data[, -1]), val_test_data$outcome)

    print(trainKMEANS)
    varImp(trainKMEANS)
    
    saveRDS(trainKMEANS, "datasets/KMEANS_001_CreditMargin.rds")
}


In [ ]:
testTmp <- modelTest %>% sample_n(size = 0.005 * nrow(modelTest))

predictKMEANS <- predict(trainKMEANS, testTmp)
summary(predictKMEANS)

cat("RMSE = ", sqrt(sum((predictKMEANS - testTmp$creditMargin)^2) / nrow(testTmp)))

In [ ]:
loansTmp <- model005 %>% select(-creditMargin, -principal_loss_pct, -monthDefault)
names(loansTmp)

In [ ]:
##############################################################
## 
## K-MEANS
## 

{
    require(doParallel)
    require(caret)
    require(cluster)
    
    tictoc::tic()
    cl <- makePSOCKcluster(4)
    registerDoParallel(cl)

    trainKMEANS <- train(x = loansTmp, y = CM005$creditMargin,
                      method="knn",
                      tuneGrid = data.frame(k = seq(30, 50, 2)),
                      trControl = trainControl(method = "cv"))

    stopCluster(cl)
    tictoc::toc()

    #confusionMatrix(predict(trainXGB, val_test_data[, -1]), val_test_data$outcome)

    print(trainKMEANS)
    varImp(trainKMEANS)
    
    saveRDS(trainKMEANS, "datasets/KMEANS_005_CreditMargin.rds")
}


In [ ]:
testTmp <- modelTest %>% sample_n(size = 0.005 * nrow(modelTest))

predictKMEANS <- predict(trainKMEANS, testTmp)
summary(predictKMEANS)

cat("RMSE = ", sqrt(sum((predictKMEANS - testTmp$creditMargin)^2) / nrow(testTmp)))

# RANDOM FOREST

Use Rborist since it accepts factors


In [ ]:
loansTmp <- model001 %>% select(-creditMargin, -principal_loss_pct, -monthDefault)
names(loansTmp)